In [1]:
import json
import torch
import os
import numpy as np
root_dir="/ltstorage/home/2pan/dataset/VG_Attribution"
annotation_file = os.path.join(root_dir, "visual_genome_attribution.json")
from datasets_zoo import snare_datasets
from torch.utils.data import DataLoader, Dataset
from clip import clip

ImportError: attempted relative import with no known parent package

In [3]:
with open(annotation_file, "r") as f:
    dataset = json.load(f)

test_dataset = snare_datasets.VG_Attribution(preprocess, attribute_ownership=True, dataset=dataset)
test_dataloader = DataLoader(test_dataset,batch_size = BATCH_SIZE, num_workers=4, shuffle=False)

In [ ]:
@torch.no_grad()
def evaluation(model, data_loader, device, validate=False):
    model.eval()

    print('Computing features for evaluation...')

    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter('eval_loss', utils.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Evaluation loss'
    print_freq = 50
    step=0
    total_loss = 0
    scores = []
    for i,batch in enumerate(metric_logger.log_every(data_loader, print_freq, header)):
    # for batch in train_dataloader :
        step+=1
        
        image_options = []
        image_embeddings = model.encode_image(batch["image_options"].to(device)).cpu()  # B x D
        image_embeddings = image_embeddings / image_embeddings.norm(dim=1, keepdim = True)
        image_options.append(np.expand_dims(image_embeddings.numpy(), axis=1))

        caption_options = []
        # print(batch['caption_options'])
        """
        [('a road with a red dirt on a small moped on a man helmet',), 
        ('a man a red helmet a a small moped on on dirt road with',), 
        ('man a a small on helmet a red with moped on dirt road a',)]

        "A man with a red helmet on a small moped on a dirt road. ", 
        
        """
        cur_loss = 0
        for idx in range(len(batch["caption_options"])):
            caption_tokenized = torch.cat([clip.tokenize(c) for c in batch["caption_options"][idx]])
            caption_embeddings = model.encode_text(caption_tokenized.to(device)).cpu()  # B x D

            # caption_embeddings = caption_embeddings - self.blank

            # caption_embeddings = caption_embeddings / np.linalg.norm(caption_embeddings, axis=1,
            #                                                             keepdims=True)  # B x D
            caption_embeddings = caption_embeddings / caption_embeddings.norm(dim=1, keepdim = True)
            caption_options.append(np.expand_dims(caption_embeddings.numpy(), axis=1))
            if validate:
                loss_func = nn.CrossEntropyLoss()
                with autocast():
                    logits_per_image = image_embeddings @ caption_embeddings.t()
                    logits_per_text = logits_per_image.T
                    # print(logits_per_image.size())
                    if idx != 1:
                        ground_truth = torch.eye(logits_per_image.size(0),device="cpu")
                        cur_loss += (loss_func(logits_per_image,ground_truth) + loss_func(logits_per_text,ground_truth))/2
                    else:
                        negative_ground_truth = torch.zeros_like(logits_per_image,device="cpu")
                        cur_loss += (loss_func(logits_per_image,negative_ground_truth) + loss_func(logits_per_text,negative_ground_truth))/2

                    total_loss += cur_loss.item()
                wandb.log({"eval_loss":cur_loss.item()})

        # print(len(caption_options))
        image_options = np.concatenate(image_options, axis=1)  # B x K x D
        # print("image_options", image_options.shape)
        caption_options = np.concatenate(caption_options, axis=1)  # B x L x D
        # print("caption_options", caption_options.shape)
        batch_scores = np.einsum("nkd,nld->nkl", image_options, caption_options)  # B x K x L


        scores.append(batch_scores)
    if validate:
        wandb.log({"eval_avg_loss":total_loss/step})
    print(metric_logger.global_avg())
    all_scores = np.concatenate(scores, axis=0)  # N x K x L
    print("all_scores", all_scores.shape)
    return all_scores

In [ ]:
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training
convert_models_to_fp32(model, True)
# print(test_dataset.all_attributes)
all_scores=evaluation(model, test_dataloader, "cuda:1", False)
if test_dataset is not None:
    test_result = test_dataset.evaluate_scores(all_scores)